In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)

c:\Users\X390\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\X390\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\X390\.cache\huggingface\hub\models--ibm--MoLFormer-XL-both-10pct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an

In [24]:
smiles = ["Cn1c(=O)c2c(ncn2C)n(C)c1=O", "CC(=O)Oc1ccccc1C(=O)O","Cn1c(=O)c2c(ncn2C)n(C)c1=O", "CC(=O)Oc1ccccc1C(=O)O"]
inputs = tokenizer(smiles, padding=True, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
outputs.pooler_output.view(-1).tolist()

[-0.44071513414382935,
 0.3902166783809662,
 0.7989372611045837,
 0.8214728236198425,
 -0.4626491963863373,
 -0.1663593202829361,
 -0.4139097034931183,
 0.4070846140384674,
 -0.3364735543727875,
 0.03279120847582817,
 -0.2715044915676117,
 0.9132962822914124,
 -0.5324793457984924,
 0.370931476354599,
 -0.001402190770022571,
 0.9211240410804749,
 -0.3484959006309509,
 -0.22645153105258942,
 -0.41305822134017944,
 -0.49036625027656555,
 -0.6671996712684631,
 0.10877116024494171,
 0.576238214969635,
 1.9819624423980713,
 0.6344061493873596,
 -0.9791179299354553,
 -0.6399291157722473,
 0.2147732526063919,
 0.7733449935913086,
 -0.2511090934276581,
 0.0177279282361269,
 -1.0294817686080933,
 -1.551544189453125,
 -0.31005486845970154,
 -0.1924687772989273,
 -0.11503227800130844,
 0.33225110173225403,
 -0.14562930166721344,
 -0.13283945620059967,
 -0.36163657903671265,
 0.27436453104019165,
 0.6969718933105469,
 -0.45629701018333435,
 0.25243932008743286,
 0.19073723256587982,
 -0.04860586673

## Load Data

In [25]:
import pickle 
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from tqdm import tqdm
import os
import torch

drug_smiles_file='../data/223drugs_pubchem_smiles.txt'
save_dir = '../data/drug_embeddings'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

def store_embeddings(drug_embeddings, save_dir, model_name):
    with open(os.path.join(save_dir, f'{model_name}.pkl'), 'wb') as f:
        pickle.dump(drug_embeddings, f)

pubchemid2smile = {item.split('\t')[0]:item.split('\t')[1].strip() for item in open(drug_smiles_file).readlines()}
model = AutoModel.from_pretrained("ibm/MoLFormer-XL-both-10pct", deterministic_eval=True, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)
model.eval()

drug_embeddings = {}
for pubchem_id, smile in tqdm(pubchemid2smile.items()):
    inputs = tokenizer(smile, padding=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.pooler_output.view(-1).tolist()
    drug_embeddings[pubchem_id] = embeddings
    #print(f"Processed {pubchem_id} with SMILES: {smile}")


store_embeddings(drug_embeddings, save_dir, 'molformer')

100%|██████████| 223/223 [00:18<00:00, 12.18it/s]


# Check Generated embeddings

In [ ]:
Drug_embedding_file = "../data/drug_embeddings/molformer.pkl"

with open(Drug_embedding_file, 'rb') as f:
    drug_feature = pickle.load(f)
drug_pubchem_id_set = set([key for key in drug_feature.keys()])

from pandas import DataFrame
DataFrame.from_dict(drug_feature, orient="index")

0         1         2    ...       765       766       767
25102847  0.056591 -0.360880  1.291251  ... -0.089176 -0.143911 -0.345757
148124    0.340644  0.152212  0.656894  ...  0.368319  0.061405  0.146496
5289247   0.247921 -0.213710  0.781715  ...  0.332639 -0.120236 -0.108908
9868037   0.365339  0.610947  0.712316  ... -0.039671 -0.119659 -0.080237
11364421 -0.477604  0.210029  0.543895  ... -0.071693 -0.115600 -0.451286
...            ...       ...       ...  ...       ...       ...       ...
24978538  0.491625  0.207371  0.887451  ... -0.031652 -0.311541 -0.874888
10461815 -0.017945  0.810465  0.661924  ...  0.126725 -0.036738 -0.290803
126565   -0.088677  0.120011  0.907875  ... -0.057634 -0.731293  0.291895
446378    0.736366 -0.051771  0.620852  ... -0.193458  0.188837 -0.348078
6253      0.461075  0.064748 -0.288596  ... -0.253373 -0.456394 -0.801219

[223 rows x 768 columns]